<a href="https://colab.research.google.com/github/hdemma/hdemma.github.io/blob/master/Macro_Prediction_Models/DataPrep_for_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geopandas
!apt install libspatialindex-dev
!pip install rtree

     |████████████████████████████████| 972kB 4.5MB/s 
     |████████████████████████████████| 10.9MB 24.6MB/s 
     |████████████████████████████████| 14.7MB 255kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ub

In [3]:
import os, time
import geopandas as gpd
import shutil
from rtree import index
from tqdm import tqdm
from shapely.ops import cascaded_union
from collections import defaultdict
import folium
from folium import plugins
from shapely.geometry import *
from IPython.display import display, HTML
import math
import random
from  more_itertools import unique_everseen
import geopy.distance
from collections import Counter
import datetime
import pandas as pd
import numpy as np
from ast import literal_eval

In [4]:
#obtain the repository
!git clone https://github.com/hdemma/hdemma.github.io.git

Cloning into 'hdemma.github.io'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 443 (delta 15), reused 0 (delta 0), pack-reused 399
Receiving objects: 100% (443/443), 75.56 MiB | 24.04 MiB/s, done.
Resolving deltas: 100% (173/173), done.


In [5]:
Vehicle_Type = 'Electric_Vehicles'
Vehicle_ID = [751]


In [6]:
%cd /content/hdemma.github.io/Dataset/OSM/
OSM_FILE ='chattanooga_osm.shp'
data = gpd.read_file(OSM_FILE)
fclass = list(data['fclass'])

CACHE_DIR = "./cache"

/content/hdemma.github.io/Dataset/OSM


In [7]:
osm_gdf = gpd.GeoDataFrame.from_file(OSM_FILE)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR)
os.makedirs(CACHE_DIR)

spatial_index_file_name = "{}/{}.spatial".format(CACHE_DIR, "osm_network")
spatial_tree = index.Index(spatial_index_file_name)

In [8]:
OSM_BUFFER = np.add(0.00015,0.00005)
GEO_Shapes_DICT = {}
osm_shps = []
with tqdm(total=len(list(osm_gdf.iterrows()))) as pbar:
    for i, gdf_row in osm_gdf.iterrows():
        shape = gdf_row['geometry']
        GEO_Shapes_DICT.update({gdf_row['osm_id']: gdf_row['geometry']})

        shape = gdf_row['geometry'].buffer(OSM_BUFFER) # Buffer it if needed.
        gdf_row['geometry'] = shape
        spatial_tree.insert(int(gdf_row["osm_id"]), shape.bounds, dict(gdf_row))
        osm_shps.append(shape)
        pbar.update(1)

#plot_shps(osm_shps).save("osm_network.html")

bounds = spatial_tree.bounds
bbox_shp = box(bounds[0],bounds[1], bounds[2], bounds[3])

100%|██████████| 27807/27807 [00:25<00:00, 1078.40it/s]


In [9]:
%cd /content/hdemma.github.io/Dataset/Electric_Vehicles/
for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')
    csv_url = 'file:///content/hdemma.github.io/Dataset/Electric_Vehicles/BYD_751_AfterEnergyCalculation.csv'

    df = pd.read_csv('BYD_751_AfterEnergyCalculation.csv', low_memory=False)

    Latitude = []
    Longitude = []
    SOC = []
    Unix_Timestamp_ms = []
    Time_US_Central = []

    empty = []
    F_Class = []
    Bridge = []
    Location_Index = []
    OSM_ID_List = []
    Street_Name = []
    OSM_ID_List_Heuristic = []
    OSM_DIST_LIST = []
    Candidate_Segment = []

    Avg_Forward_dist_LIST = []
    Avg_Backward_dist_LIST = []
    Max_Forward_dist_LIST = []
    Max_Backward_dist_LIST = []
    Min_Forward_dist_LIST = []
    Min_Backward_dist_LIST = []

    osm_geom_dict = {}


    unix_Timestamp_ms = list(df['Unix Timestamp [ms]'])
    time_US_Central = list(df['Time [US/Central]'])
    soc = list(df['Computed_SoC_%'])
    Lat = list(df['GPS - GPS position Latitude'])
    Long = list(df['GPS - GPS position Longitude'])

    
    Road_Types_to_AVOID = [ 'path', 'pedestrian', 'footway', 'steps', 'cycleway','bridleway','unknown']


    with tqdm(total=len(Lat)) as p:
        for i in range(0,len(Lat)):
            Lat[i] = float(Lat[i])
            Long[i] = float(Long[i])
            Location_Index.append(i)
            SOC.append(soc[i])
            Unix_Timestamp_ms.append(unix_Timestamp_ms[i])
            Time_US_Central.append(time_US_Central[i])
            if not math.isnan(Lat[i]):
                osm_id_individual = {}
 
                Point_Shape = Point(Long[i], Lat[i])
                Latitude.append(Lat[i])
                Longitude.append(Long[i])


                #Temp for each location
                for_heuristic =[]
                osm = []
                street_name = []
                dist = []
                fclass = []
                bridge = []
                Avg_dist = []
                geometry = []

                Avg_Forward = []
                Avg_Backward = []
                Max_Forward = []
                Max_Backward = []
                Min_Forward = []
                Min_Backward = []

                candidate = []

                Point_buffered_Shape = Point(Long[i], Lat[i])  # .buffer(0.00001)

                results = list(spatial_tree.intersection(Point_buffered_Shape.bounds, objects='raw'))
                results = [street for street in results if street["geometry"].intersection(Point_buffered_Shape)]
                FinalSelection_Streets = [street for street in results if street["fclass"] not in Road_Types_to_AVOID]

                if len(FinalSelection_Streets)>=1:
                    candidate_Number = 0
                    for FS in FinalSelection_Streets:

                        G = FS['geometry']
                        candidate_Number+=1
                        candidate.append(candidate_Number)

                        

                        osm.append(FS['osm_id'])
                        street_name.append(FS['name'])
                        geometry.append(FS['geometry'])

                        distance_between_pts = G.exterior.distance(Point_Shape)
                        dist.append(distance_between_pts)

                        fclass.append(FS['fclass'])

                        if FS['bridge'] == 'T':
                            bridge.append(1)
                        else:
                            bridge.append(0)

                        if i == 0:
                            Avg_Forward.append(empty)
                            Avg_Backward.append(empty)

                            Max_Forward.append(empty)
                            Max_Backward.append(empty)

                            Min_Forward.append(empty)
                            Min_Backward.append(empty)

                        else:
                            if i>0 and i< 15:
                                tmp_forward = []
                                tmp_backward = []
                                for j in range(len(Lat[i:i + 14])):
                                    Point_Shape = Point(Long[j], Lat[j])
                                    distance_between_pts = G.exterior.distance(Point_Shape)

                                    tmp_forward.append(distance_between_pts)

                                Avg_Forward.append(sum(tmp_forward) / len(tmp_forward))
                                Max_Forward.append(max(tmp_forward))
                                Min_Forward.append(min(tmp_forward))

                                Avg_Backward.append(empty)
                                Max_Backward.append(empty)
                                Min_Backward.append(empty)

                            else:
                                tmp_forward = []
                                tmp_backward = []
                                for j in range(len(Lat[i - 15: i])):
                                    Point_Shape = Point(Long[j], Lat[j])
                                    distance_between_pts = G.exterior.distance(Point_Shape)

                                    tmp_backward.append(distance_between_pts)

                                for j in range(len(Lat[i: i + 16])):
                                    Point_Shape = Point(Long[j], Lat[j])
                                    distance_between_pts = G.exterior.distance(Point_Shape)

                                    tmp_forward.append(distance_between_pts)

                                Avg_Forward.append(sum(tmp_forward)/len(tmp_forward))
                                Avg_Backward.append(sum(tmp_backward)/len(tmp_backward))

                                Max_Forward.append(max(tmp_forward))
                                Max_Backward.append(max(tmp_backward))

                                Min_Forward.append(min(tmp_forward))
                                Min_Backward.append(min(tmp_backward))


                osm_geom_dict= {osm_feature["osm_id"]: osm_feature for osm_feature in FinalSelection_Streets}


                if len(osm_geom_dict) != 0:
                    for k, v in osm_geom_dict.items():
                        osm_id_individual.update({k:v})
                    for_heuristic.append(osm_id_individual) # add the value

                    OSM_ID_List_Heuristic.append(for_heuristic)
                    OSM_ID_List.append(osm)
                    Street_Name.append(street_name)
                    OSM_DIST_LIST.append(dist)
                    F_Class.append(fclass)
                    Bridge.append(bridge)

                    Avg_Forward_dist_LIST.append(Avg_Forward)
                    Avg_Backward_dist_LIST.append(Avg_Backward)

                    Max_Forward_dist_LIST.append(Max_Forward)
                    Max_Backward_dist_LIST.append(Max_Backward)

                    Min_Forward_dist_LIST.append(Min_Forward)
                    Min_Backward_dist_LIST.append(Min_Backward)
                    Candidate_Segment.append(candidate)
                else:
                    OSM_ID_List.append(empty)
                    OSM_ID_List_Heuristic.append(empty)
                    Street_Name.append(empty)

                    OSM_DIST_LIST.append(empty)
                    Avg_Forward_dist_LIST.append(empty)
                    Avg_Backward_dist_LIST.append(empty)

                    Max_Forward_dist_LIST.append(empty)
                    Max_Backward_dist_LIST.append(empty)

                    Min_Forward_dist_LIST.append(empty)
                    Min_Backward_dist_LIST.append(empty)

                    F_Class.append(empty)
                    Bridge.append(empty)

                    Candidate_Segment.append(empty)

            else:
                Latitude.append(Lat[i])
                Longitude.append(Long[i])

                OSM_ID_List.append(empty)
                OSM_ID_List_Heuristic.append(empty)
                Street_Name.append(empty)

                OSM_DIST_LIST.append(empty)
                Avg_Forward_dist_LIST.append(empty)
                Avg_Backward_dist_LIST.append(empty)

                Max_Forward_dist_LIST.append(empty)
                Max_Backward_dist_LIST.append(empty)

                Min_Forward_dist_LIST.append(empty)
                Min_Backward_dist_LIST.append(empty)
                F_Class.append(empty)
                Bridge.append(empty)
                Candidate_Segment.append(empty)

            p.update(1)

    Final_value = []
    Final_Count = []
    print(len(OSM_ID_List_Heuristic))
    with tqdm(total = len(Lat)) as pbar:
        for i in range(0, len(Lat)):
            if not math.isnan(Lat[i]):
                Point_Shape = Point(Long[i], Lat[i])
                test_shp = Point(Long[i], Lat[i])  # .buffer(0.002)

                if len(OSM_ID_List_Heuristic[i]) != 0:
                    count = []
                    ID = []
                    tmp_ID_result = []
                    c = []
                    for street in OSM_ID_List_Heuristic[i]:
                        for key, value in street.items():
                            ID.append(key)
                            tmp_ID_result.append(0)
                            for key2, value2 in value.items():  # Nested Dictionary
                                if key2 == 'fclass':
                                    if value2 == 'unclassified':
                                        c.append(-2)
                                    else:
                                        c.append(0)
                    for a in range(len(ID)):
                        if i<15:
                            for j in OSM_ID_List_Heuristic[:15]:
                                if len(j) != 0:
                                    for b in j:
                                        for key, value in b.items():
                                            if ID[a] == key:
                                                c[a] += 1
                                else:
                                    continue

                            count.append(c[a])
                        else:
                            for j in OSM_ID_List_Heuristic[i - 15: i + 16]:
                                if len(j)!= 0:
                                    for b in j:
                                        for key, value in b.items():
                                            if ID[a] == key:
                                                c[a] += 1
                                else:
                                    continue

                            count.append(c[a])

                    m = count.index(max(count))

                    tmp_ID_result[m] = 1
                    Final_Count.append(count)
                    Final_value.append(tmp_ID_result)


                else:
                    Final_value.append(empty)
                    Final_Count.append(empty)


            else:
                Final_value.append(empty)
                Final_Count.append(empty)

            pbar.update(1)


    df = pd.DataFrame({"Location_Index":Location_Index,
                       "Unix_Timestamp_ms":Unix_Timestamp_ms,
                       "Time_US_Central":Time_US_Central,
                       "SOC":SOC,
                        "Latitude":Latitude,
                       "Longitude": Longitude,
                       "Candidate":Candidate_Segment,
                       "OSM_ID":OSM_ID_List,
                       "Street_Name":Street_Name,
                       "F_Class":F_Class,
                       "Distance_from_location":OSM_DIST_LIST,
                       "Average_Forward_Location":Avg_Forward_dist_LIST,
                       "Average_Backward_Location":Avg_Backward_dist_LIST,
                       "Max_Forward_Location":Max_Forward_dist_LIST,
                       "Max_Backward_Location": Max_Backward_dist_LIST,
                       "Min_Forward_Location":Min_Forward_dist_LIST,
                       "Min_Backward_Location": Min_Backward_dist_LIST,
                       "Bridge":Bridge,
                       "Final_Count":Final_Count,
                       "Final_Output": Final_value
                       })

    csv = f"{Vehicle_Name}_Data_for_Map_Matching.csv"
    df.to_csv(csv, index=False)


/content/hdemma.github.io/Dataset/Electric_Vehicles
Processing BYD_751


  2%|▏         | 1451/82479 [00:00<00:05, 14507.73it/s]

82479


100%|██████████| 82479/82479 [00:05<00:00, 14083.91it/s]


Data Splitting (One row for each candidate)

In [12]:
%cd /content/hdemma.github.io/Dataset/Electric_Vehicles/
import tqdm
for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'

    csv = f"{Vehicle_Name}_Data_for_Map_Matching.csv"
    df = pd.read_csv(csv, low_memory=False)

    print(df.columns)

    Columns = ['Candidate', 'OSM_ID', 'Street_Name','F_Class',
               'Distance_from_location', 'Average_Forward_Location',
               'Average_Backward_Location', 'Max_Forward_Location',
               'Max_Backward_Location', 'Min_Forward_Location',
               'Min_Backward_Location', 'Bridge', 'Final_Count','Final_Output']

    for col in df.columns:
        if col in Columns:
            df[col] = df[col].apply(literal_eval)

    unix_Timestamp_ms = list(df.Unix_Timestamp_ms)
    time_US_Central = list(df.Time_US_Central)
    location_index = list(df.Location_Index)
    latitude = list(df.Latitude)
    longitude = list(df.Longitude)
    soc = df.SOC
    candidate = list(df.Candidate)
    fclass = list(df.F_Class)
    osm_id = list(df.OSM_ID)
    street_name = list(df.Street_Name)
    distance_from_location = list(df.Distance_from_location)
    average_forward_location = list(df.Average_Forward_Location)
    average_backward_location = list(df.Average_Backward_Location)
    max_forward_location = list(df.Max_Forward_Location)
    max_backward_location = list(df.Max_Backward_Location)
    min_forward_location = list(df.Min_Forward_Location)
    min_backward_location = list(df.Min_Backward_Location)
    bridge = list(df.Bridge)
    final_count = list(df.Final_Count)
    final_output = list(df.Final_Output)

    Location_Index = []
    Latitude = []
    Longitude = []
    SOC = []
    Unix_Timestamp_ms = []
    Time_US_Central = []
    Candidate = []
    F_Class = []
    OSM_ID = []
    Street_Name = []

    Distance_from_location = []
    Average_Forward_Location = []
    Average_Backward_Location = []
    Max_Forward_Location = []
    Max_Backward_Location = []
    Min_Forward_Location = []
    Min_Backward_Location = []
    Bridge = []
    Final_Count = []
    Final_Output = []


    for i in tqdm.tqdm(range(len(df))):
        for j in range(len(candidate[i])):
            Latitude.append(latitude[i])
            Longitude.append(longitude[i])
            SOC.append(soc[i])
            Unix_Timestamp_ms.append(unix_Timestamp_ms[i])
            Time_US_Central.append(time_US_Central[i])
            Location_Index.append(location_index[i])

            if average_forward_location[i][j]==[]:
                Average_Forward_Location.append('')
            else:
                Average_Forward_Location.append(average_forward_location[i][j])

            if average_backward_location[i][j]== []:
                Average_Backward_Location.append('')
            else:
                Average_Backward_Location.append(average_backward_location[i][j])

            if max_forward_location[i][j]==[]:
                Max_Forward_Location.append('')
            else:
                Max_Forward_Location.append(max_forward_location[i][j])

            if max_backward_location[i][j]==[]:
                Max_Backward_Location.append('')
            else:
                Max_Backward_Location.append(max_backward_location[i][j])

            if min_forward_location[i][j]==[]:
                Min_Forward_Location.append('')
            else:
                Min_Forward_Location.append(min_forward_location[i][j])

            if min_backward_location[i][j]==[]:
                Min_Backward_Location.append('')
            else:
                Min_Backward_Location.append(min_backward_location[i][j])
            Candidate.append(candidate[i][j])

            OSM_ID.append(osm_id[i][j])
            F_Class.append(fclass[i][j])
            Street_Name.append(street_name[i][j])

            Distance_from_location.append(distance_from_location[i][j])

            Bridge.append(bridge[i][j])
            Final_Count.append(final_count[i][j])
            Final_Output.append(final_output[i][j])


    Road_Type = ['primary', 'primary_link',
                 'secondary','secondary_link',
                 'tertiary', 'tertiary_link',
                  'trunk',
                 'motorway','motorway_link',
                 'service', 'residential',
                 'track',
                'unknown', 'unclassified']

    primary=[]
    primary_link=[]
    secondary=[]
    secondary_link=[]
    tertiary=[]
    tertiary_link=[]
    trunk=[]
    motorway=[]
    motorway_link=[]
    service=[]
    residential=[]
    track=[]
    unknown = []
    unclassified=[]


    for i in range(len(F_Class)):
        RT = []
        for roadtype in range(len(Road_Type)):
            if Road_Type[roadtype] == F_Class[i]:


                RT.append(1)
            else:
                RT.append(0)
        # print(RT)
        for xx in range(len(RT)):
            if xx == 0:
                primary.append(RT[xx])
            elif xx == 1:
                primary_link.append(RT[xx])
            elif xx == 2:
                secondary.append(RT[xx])
            elif xx == 3:
                secondary_link.append(RT[xx])
            elif xx == 4:
                tertiary.append(RT[xx])
            elif xx == 5:
                tertiary_link.append(RT[xx])
            elif xx == 6:
                trunk.append(RT[xx])
            elif xx == 7:
                motorway.append(RT[xx])
            elif xx == 8:
                motorway_link.append(RT[xx])
            elif xx == 9:
                service.append(RT[xx])
            elif xx == 10:
                residential.append(RT[xx])
            elif xx == 11:
                track.append(RT[xx])
            elif xx == 12:
                unknown.append(RT[xx])
            elif xx == 13:
                unclassified.append(RT[xx])


    df = pd.DataFrame({"Location_Index":Location_Index,
                       "Latitude": Latitude,
                       "Longitude": Longitude,
                        "Unix_Timestamp_ms":Unix_Timestamp_ms,
                       "Time_US_Central":Time_US_Central,
                       "SOC":SOC,
                       "Candidate": Candidate,
                       "OSM_ID": OSM_ID,
                       "Street_Name": Street_Name,
                       "F_Class": F_Class,
                       "Distance_from_location": Distance_from_location,
                       "Average_Forward_Location": Average_Forward_Location,
                       "Average_Backward_Location": Average_Backward_Location,
                       "Max_Forward_Location": Max_Forward_Location,
                       "Max_Backward_Location": Max_Backward_Location,
                       "Min_Forward_Location": Min_Forward_Location,
                       "Min_Backward_Location": Min_Backward_Location,
                       "Bridge": Bridge,
                       "Primary":primary,
                       "Primary_link":primary_link,
                       "Secondary":secondary,
                       "Secondary_link":secondary_link,
                       "Tertiary":tertiary,
                       "Tertiary_link":tertiary_link,
                       "Trunk":trunk,
                       "Motorway":motorway,
                       "Motorway_link":motorway_link,
                       "Service":service,
                       "Residential":residential,
                       "Track":track,
                       "Unknown":unknown,
                       "Unclassified":unclassified,
                       "Final_Count":Final_Count,
                       "Final_Output": Final_Output
                       })

    csv = f"{Vehicle_Name}_Data_for_Map_Matching_AfterSplit.csv"
    df.to_csv(csv, index=False)


/content/hdemma.github.io/Dataset/Electric_Vehicles
Index(['Location_Index', 'Unix_Timestamp_ms', 'Time_US_Central', 'SOC',
       'Latitude', 'Longitude', 'Candidate', 'OSM_ID', 'Street_Name',
       'F_Class', 'Distance_from_location', 'Average_Forward_Location',
       'Average_Backward_Location', 'Max_Forward_Location',
       'Max_Backward_Location', 'Min_Forward_Location',
       'Min_Backward_Location', 'Bridge', 'Final_Count', 'Final_Output'],
      dtype='object')


100%|██████████| 82479/82479 [00:02<00:00, 29349.54it/s]
